In [18]:
import cv2
import numpy as np
import glob
import yaml

# Configuration
GRID_SIZE = (7, 9)  # Inner corners (rows-1, cols-1) for a 7x9 checkerboard
SQUARE_DIM = 0.015  # 15mm in meters (square size)
LEFT_IMAGES_PATH = r"Dataset_6\left\*.png"  # Path to left camera images (use raw string)
RIGHT_IMAGES_PATH = r"Dataset_6\right\*.png"  # Path to right camera images (use raw string)
IMAGE_DIM = (1280, 720)  # (width, height) of images

# Set up object points (3D coordinates of the checkerboard corners)
object_points = np.zeros((GRID_SIZE[0] * GRID_SIZE[1], 3), np.float32)
object_points[:, :2] = np.mgrid[0:GRID_SIZE[0], 0:GRID_SIZE[1]].T.reshape(-1, 2) * SQUARE_DIM

def perform_camera_calibration(image_paths, camera_name):
    """Detect corners in checkerboard images and calculate camera intrinsic parameters."""
    object_points_list, image_points_list = [], []
    valid_files, invalid_files = [], []
    
    image_files = sorted(glob.glob(image_paths))
    if not image_files:
        raise ValueError(f"No images found at {image_paths}!")

    print(f"\n=== Processing {camera_name} ===")
    print(f"Images found: {len(image_files)}")
    
    for file_index, file_path in enumerate(image_files, 1):
        image = cv2.imread(file_path)
        if image is None:
            invalid_files.append(file_path)
            continue
            
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        found, corners = cv2.findChessboardCorners(
            grayscale_image, GRID_SIZE,
            cv2.CALIB_CB_ADAPTIVE_THRESH + 
            cv2.CALIB_CB_FAST_CHECK +
            cv2.CALIB_CB_NORMALIZE_IMAGE
        )
        
        if found:
            refined_corners = cv2.cornerSubPix(grayscale_image, corners, (11, 11), (-1, -1),
                                               (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            
            object_points_list.append(object_points)
            image_points_list.append(refined_corners)
            valid_files.append(file_path)
        else:
            invalid_files.append(file_path)

    if not object_points_list:
        raise ValueError(f"No valid images found for {camera_name}!")

    # Camera calibration
    retval, camera_matrix, dist_coeffs, rotation_vectors, translation_vectors = cv2.calibrateCamera(
        object_points_list, image_points_list, grayscale_image.shape[::-1], None, None,
        flags=cv2.CALIB_ZERO_TANGENT_DIST + cv2.CALIB_FIX_K3 + cv2.CALIB_FIX_K4 + cv2.CALIB_FIX_K5
    )
    
    # Compute the reprojection error
    total_error = 0
    for i in range(len(object_points_list)):
        projected_points, _ = cv2.projectPoints(object_points_list[i], rotation_vectors[i], translation_vectors[i], camera_matrix, dist_coeffs)
        error = cv2.norm(image_points_list[i], projected_points, cv2.NORM_L2) / len(projected_points)
        total_error += error
    mean_error = total_error / len(object_points_list)
    
    return camera_matrix, dist_coeffs, object_points_list, image_points_list, mean_error, image_files

# Function to convert numpy arrays and tuples into lists for saving to YAML
def convert_to_list(data):
    if isinstance(data, np.ndarray):
        return data.tolist()  # Convert numpy array to list
    elif isinstance(data, tuple):
        return list(data)  # Convert tuple to list
    elif isinstance(data, list):
        return [convert_to_list(item) for item in data]  # Recursively handle lists
    else:
        return data  # Return other types as is

# Main stereo calibration process
try:
    print("Initiating stereo calibration...")
    left_camera_matrix, left_dist_coeffs, left_objpoints, left_imgpoints, left_error, _ = perform_camera_calibration(LEFT_IMAGES_PATH, "Left Camera")
    right_camera_matrix, right_dist_coeffs, right_objpoints, right_imgpoints, right_error, _ = perform_camera_calibration(RIGHT_IMAGES_PATH, "Right Camera")

    # Stereo calibration
    calibration_flags = cv2.CALIB_FIX_INTRINSIC + cv2.CALIB_USE_INTRINSIC_GUESS
    termination_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)

    retval, K1, D1, K2, D2, rotation_matrix, translation_vector, essential_matrix, fundamental_matrix = cv2.stereoCalibrate(
        left_objpoints, left_imgpoints, right_imgpoints,
        left_camera_matrix, left_dist_coeffs, right_camera_matrix, right_dist_coeffs,
        IMAGE_DIM,
        criteria=termination_criteria, flags=calibration_flags)

    # Perform stereo rectification
    R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
        K1, D1, K2, D2, 
        IMAGE_DIM,  # Fixed image size
        rotation_matrix, translation_vector, 
        flags=cv2.CALIB_ZERO_DISPARITY, 
        alpha=0.9
    )

    # Saving calibration results
    calibration_results = {
        'image_size': IMAGE_DIM,
        'K_left': left_camera_matrix, 'D_left': left_dist_coeffs,
        'K_right': right_camera_matrix, 'D_right': right_dist_coeffs,
        'R': rotation_matrix, 'T': translation_vector,
        'E': essential_matrix, 'F': fundamental_matrix,
        'R1': R1, 'R2': R2,
        'P1': P1, 'P2': P2,
        'Q': Q,
        'error_left': left_error,
        'error_right': right_error,
        'stereo_error': retval
    }

    # Convert numpy arrays and tuples into lists for saving in YAML
    calibration_results = {k: convert_to_list(v) for k, v in calibration_results.items()}

    # Save results to YAML file
    with open('stereo_calibration_results.yaml', 'w') as f:
        yaml.dump(calibration_results, f)

    print("Stereo calibration and rectification successful.")
    print("Results saved to stereo_calibration_results.yaml")

    # Loading and displaying results from YAML
    with open('stereo_calibration_results.yaml', 'r') as f:
        results = yaml.safe_load(f)

    def print_matrix(matrix_name, matrix):
        print(f"\n{matrix_name}:")
        for row in matrix:
            print("  [", ", ".join(f"{x:.8f}" for x in row), "]")

    # Display the left camera calibration results
    print("=== Left Camera Calibration Results ===")
    print(f"Mean Reprojection Error: {results['error_left']:.5f} px")
    print_matrix("Camera Matrix", results['K_left'])
    print("Distortion Coefficients:", np.array(results['D_left']).ravel())

    # Display the right camera calibration results
    print("\n=== Right Camera Calibration Results ===")
    print(f"Mean Reprojection Error: {results['error_right']:.5f} px")
    print_matrix("Camera Matrix", results['K_right'])
    print("Distortion Coefficients:", np.array(results['D_right']).ravel())

    # Display stereo calibration results
    print("\n=== Stereo Calibration Results ===")
    print(f"Stereo Reprojection Error: {results['stereo_error']:.5f}")
    print_matrix("Rotation Matrix (R)", results['R'])
    print_matrix("Translation Vector (T)", results['T'])
    print_matrix("Rectification Rotation R1", results['R1'])
    print_matrix("Rectification Rotation R2", results['R2'])
    print_matrix("Projection Matrix P1", results['P1'])
    print_matrix("Projection Matrix P2", results['P2'])
    print_matrix("Disparity-to-Depth Matrix Q", results['Q'])
    print("\nImage Size:", tuple(results['image_size']))

except Exception as e:
    print(f"Calibration failed: {str(e)}")

Initiating stereo calibration...

=== Processing Left Camera ===
Images found: 57

=== Processing Right Camera ===
Images found: 57
Stereo calibration and rectification successful.
Results saved to stereo_calibration_results.yaml
=== Left Camera Calibration Results ===
Mean Reprojection Error: 0.02066 px

Camera Matrix:
  [ 1059.82779022, 0.00000000, 646.18224656 ]
  [ 0.00000000, 1059.39069888, 405.35767342 ]
  [ 0.00000000, 0.00000000, 1.00000000 ]
Distortion Coefficients: [ 0.0335875  -0.38226424  0.          0.          0.        ]

=== Right Camera Calibration Results ===
Mean Reprojection Error: 0.01956 px

Camera Matrix:
  [ 1222.90688624, 0.00000000, 635.64289995 ]
  [ 0.00000000, 1220.45822800, 354.18104589 ]
  [ 0.00000000, 0.00000000, 1.00000000 ]
Distortion Coefficients: [ 0.06259462 -0.86894772  0.          0.          0.        ]

=== Stereo Calibration Results ===
Stereo Reprojection Error: 45.88093

Rotation Matrix (R):
  [ 0.99753504, -0.03532767, 0.06062839 ]
  [ 0.03